In [16]:
from google.colab import drive
drive.mount('/drive', force_remount=True)

Mounted at /drive
time: 1.09 s (started: 2021-06-12 15:06:13 +00:00)


In [17]:
!pip install ipython-autotime
%load_ext autotime
import pickle
import os
import time
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.externals import joblib
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (20,10)

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 2.78 s (started: 2021-06-12 15:06:14 +00:00)


In [18]:
import tensorflow as tf
print("Using Keras",tf.keras.__version__)

Using Keras 2.5.0
time: 1.24 ms (started: 2021-06-12 15:06:16 +00:00)


In [19]:
window_length = 30
timeframe = "15minutes"
classes = 2
candles = 1
network = None
scaler = None
batch_size = 128

layer_tune = [1, 2]
neurons_tune = [64, 128]
dropout_tune = [0.5]
lr_tune = [0.001, 0.0001, 0.00001]
tune_results_file = "/drive/My Drive/disertation/tunning_results_lstm_{}_window_{}.csv".format(window_length, timeframe)

time: 6.22 ms (started: 2021-06-12 15:06:16 +00:00)


In [20]:
def extend_dataset_with_window_length(X, Y, window_length=window_length):
    new_x = []
    for i in range(len(X) - window_length+1):
        lst = []
        for j in range(i, i+window_length):
            lst.extend(X[j])
        new_x.append(lst)
    return np.array(new_x), Y[window_length-1:]

def get_equal_class_distribution(X,Y):
    ys = pd.Series(Y)
    vs = ys.value_counts()
    required_of_each_class = vs.min()
    original_indexes = []
    for idx in vs.index:
        original_indexes.extend(ys[ys==idx].sample(n=required_of_each_class, replace=False, random_state=1).index)
    return X[original_indexes], Y[original_indexes]

time: 9.2 ms (started: 2021-06-12 15:06:16 +00:00)


In [21]:
train_df = pd.read_csv("/drive/My Drive/disertation/train_df_{}_{}_candles_{}_class.csv".format(timeframe, candles, classes))
train_df = train_df.set_index("open_time")
train_df.drop(columns=["close"],inplace=True)
train_df.index = pd.to_datetime(train_df.index)

data = train_df.to_numpy()
X = data[:,:-1]
Y = data[:,-1]

X, Y = extend_dataset_with_window_length(X,Y)

time: 4.78 s (started: 2021-06-12 15:06:17 +00:00)


In [22]:
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

scaler_path = "/drive/My Drive/disertation/lstms/lstm_scaler_{}_{}window.save".format(timeframe, window_length)
joblib.dump(scaler, scaler_path)

['/drive/My Drive/disertation/lstms/lstm_scaler_15minutes_30window.save']

time: 179 ms (started: 2021-06-12 15:06:21 +00:00)


In [23]:
a,b = np.unique(Y, return_counts=True)
print("buy sell %")
print(b)
print(b / sum(b))

X,Y = get_equal_class_distribution(X,Y)

a,b = np.unique(Y, return_counts=True)
print("buy sell %")
print(b)
print(b / sum(b))

n_features = X.shape[1] // window_length
n_timesteps = window_length
X = X.reshape(X.shape[0], n_timesteps, n_features)

buy sell %
[10095  9076]
[0.5265766 0.4734234]
buy sell %
[9076 9076]
[0.5 0.5]
time: 61 ms (started: 2021-06-12 15:06:21 +00:00)


In [24]:
tune_df = pd.read_csv("/drive/My Drive/disertation/tune_df_{}_{}_candles_{}_class.csv".format(timeframe, candles, classes))
tune_df = tune_df.set_index("open_time")
tune_df.drop(columns=["close"],inplace=True)

data = tune_df.to_numpy()
X_tune = data[:,:-1]
Y_tune = data[:,-1]

X_tune, Y_tune = extend_dataset_with_window_length(X_tune, Y_tune)

X_tune = scaler.transform(X_tune)
X_tune = X_tune.reshape(X_tune.shape[0], n_timesteps, n_features)


test_df = pd.read_csv("/drive/My Drive/disertation/test_df_{}_{}_candles_{}_class.csv".format(timeframe, candles, classes))
test_df = test_df.set_index("open_time")
test_df.drop(columns=["close"],inplace=True)

data = test_df.to_numpy()
X_test = data[:,:-1]
Y_test = data[:,-1]

X_test, Y_test = extend_dataset_with_window_length(X_test, Y_test)

X_test = scaler.transform(X_test)
X_test = X_test.reshape(X_test.shape[0], n_timesteps, n_features)


time: 8.78 s (started: 2021-06-12 15:06:22 +00:00)


In [25]:
a,b = np.unique(Y_tune, return_counts=True)
print("buy sell %")
print(b)
print(b / sum(b))
tune_buy_percentage = (b/sum(b))[0]

buy sell %
[9757 9414]
[0.5089458 0.4910542]
time: 10.2 ms (started: 2021-06-12 15:06:30 +00:00)


In [26]:
a,b = np.unique(Y_test, return_counts=True)
print("buy sell %")
print(b)
print(b / sum(b))
test_buy_percentage = (b/sum(b))[0]

buy sell %
[9657 9514]
[0.50372959 0.49627041]
time: 6.51 ms (started: 2021-06-12 15:06:30 +00:00)


In [27]:
def create_model(layers_count, neurons, lr, dropout):
    network = models.Sequential()
    if layers_count == 1:
        network.add(layers.LSTM(neurons, input_shape=(n_timesteps, n_features)))
        network.add(layers.Dropout(dropout))
    elif layers_count == 2:
        network.add(layers.LSTM(neurons, input_shape=(n_timesteps, n_features), return_sequences=True))
        network.add(layers.Dropout(dropout))
        network.add(layers.LSTM(neurons))
        network.add(layers.Dropout(dropout))

    network.add(layers.Dense(neurons, activation='relu'))

    network.add(layers.Dense(1, activation="sigmoid"))
    loss = "binary_crossentropy"
    opt = tf.keras.optimizers.Adam(learning_rate=lr)
    network.compile(optimizer='adam', loss=loss, metrics=['accuracy'])

    # print(network.summary())

    return network

time: 15.2 ms (started: 2021-06-12 15:06:30 +00:00)


In [28]:
def train_network(network):
    callback_early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=20, restore_best_weights=True)
    history = network.fit(X, Y, epochs=400, batch_size=batch_size, validation_data=(X_tune, Y_tune), callbacks=[callback_early_stop])
    return history
    # pd.DataFrame(history.history).plot(lw=2);

time: 3.99 ms (started: 2021-06-12 15:06:30 +00:00)


In [29]:
results = []

attempt_number = 0
total_attempts = len(layer_tune) * len(neurons_tune) * len(lr_tune) * len(dropout_tune)

for layers_count in layer_tune:
    for neurons in neurons_tune:
        for lr in lr_tune:
            for dropout in dropout_tune:
                attempt_number += 1
                print("Attempt {}/{}".format(attempt_number, total_attempts))

                network = create_model(layers_count=layers_count, neurons=neurons, lr=lr, dropout=dropout)
                t = time.time()
                history = train_network(network)
                train_time = time.time() - t

                model_file = "/drive/My Drive/disertation/lstms/lstm_{}__{}_{}window.h5".format(attempt_number, timeframe, window_length)

                if os.path.exists(model_file):
                    raise Exception("file exists!")
                network.save(model_file)

                Y_predicted = network.predict_classes(X)
                report = metrics.classification_report(Y, Y_predicted, digits=3, output_dict=True)
                f1_train = report['weighted avg']['f1-score']
                acc_train = report["accuracy"]

                Y_tune_predicted = network.predict_classes(X_tune)
                report = metrics.classification_report(Y_tune, Y_tune_predicted, digits=3, output_dict=True)
                f1_tune = report['weighted avg']['f1-score']
                acc_tune = report["accuracy"]

                Y_test_predicted = network.predict_classes(X_test)
                report = metrics.classification_report(Y_test, Y_test_predicted, digits=3, output_dict=True)
                f1_test = report['weighted avg']['f1-score']
                acc_test = report['accuracy']

                r = {
                    "attempt": attempt_number,
                    "layers": layers_count,
                    "neurons": neurons,
                    "lr": lr,
                    "train_time": train_time,
                    "epochs": len(history.epoch),
                    "train_acc": acc_train,
                    "train_f1": f1_train,
                    "tune_acc": acc_tune,
                    "tune_f1": f1_tune,
                    "test_acc": acc_test,
                    "test_f1": f1_test,
                    "tune_buy_percentage": tune_buy_percentage,
                    "test_buy_percentage": test_buy_percentage
                }
                results.append(r)
                print(pd.DataFrame(results)[["train_acc", "train_f1", "tune_acc", "tune_f1", "test_acc", "test_f1"]])

Attempt 1/12
Epoch 1/400
142/142 [==============================] - 9s 49ms/step - loss: 0.6949 - accuracy: 0.5012 - val_loss: 0.6925 - val_accuracy: 0.5188
Epoch 2/400
142/142 [==============================] - 6s 44ms/step - loss: 0.6937 - accuracy: 0.5069 - val_loss: 0.6922 - val_accuracy: 0.5172
Epoch 3/400
142/142 [==============================] - 6s 44ms/step - loss: 0.6933 - accuracy: 0.5086 - val_loss: 0.6927 - val_accuracy: 0.5155
Epoch 4/400
142/142 [==============================] - 6s 44ms/step - loss: 0.6933 - accuracy: 0.5046 - val_loss: 0.6923 - val_accuracy: 0.5181
Epoch 5/400
142/142 [==============================] - 6s 45ms/step - loss: 0.6925 - accuracy: 0.5117 - val_loss: 0.6926 - val_accuracy: 0.5115
Epoch 6/400
142/142 [==============================] - 6s 44ms/step - loss: 0.6926 - accuracy: 0.5127 - val_loss: 0.6919 - val_accuracy: 0.5192
Epoch 7/400
142/142 [==============================] - 6s 44ms/step - loss: 0.6919 - accuracy: 0.5131 - val_loss: 0.6920 - 

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


   train_acc  train_f1  tune_acc   tune_f1  test_acc   test_f1
0   0.542364  0.542234  0.540139  0.537295  0.526994  0.525064
Attempt 2/12
Epoch 1/400
142/142 [==============================] - 8s 47ms/step - loss: 0.6957 - accuracy: 0.5025 - val_loss: 0.6927 - val_accuracy: 0.5129
Epoch 2/400
142/142 [==============================] - 6s 43ms/step - loss: 0.6940 - accuracy: 0.5017 - val_loss: 0.6929 - val_accuracy: 0.5087
Epoch 3/400
142/142 [==============================] - 6s 43ms/step - loss: 0.6934 - accuracy: 0.5021 - val_loss: 0.6928 - val_accuracy: 0.5119
Epoch 4/400
142/142 [==============================] - 7s 47ms/step - loss: 0.6934 - accuracy: 0.5049 - val_loss: 0.6928 - val_accuracy: 0.5126
Epoch 5/400
142/142 [==============================] - 6s 43ms/step - loss: 0.6929 - accuracy: 0.5083 - val_loss: 0.6924 - val_accuracy: 0.5196
Epoch 6/400
142/142 [==============================] - 6s 43ms/step - loss: 0.6931 - accuracy: 0.5088 - val_loss: 0.6922 - val_accuracy: 0.52

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


   train_acc  train_f1  tune_acc   tune_f1  test_acc   test_f1
0   0.542364  0.542234  0.540139  0.537295  0.526994  0.525064
1   0.534762  0.533756  0.541808  0.541606  0.519326  0.516555
Attempt 3/12
Epoch 1/400
142/142 [==============================] - 8s 47ms/step - loss: 0.6951 - accuracy: 0.5023 - val_loss: 0.6932 - val_accuracy: 0.5089
Epoch 2/400
142/142 [==============================] - 6s 44ms/step - loss: 0.6939 - accuracy: 0.5051 - val_loss: 0.6927 - val_accuracy: 0.5087
Epoch 3/400
142/142 [==============================] - 6s 43ms/step - loss: 0.6932 - accuracy: 0.5056 - val_loss: 0.6927 - val_accuracy: 0.5207
Epoch 4/400
142/142 [==============================] - 6s 43ms/step - loss: 0.6930 - accuracy: 0.5121 - val_loss: 0.6932 - val_accuracy: 0.4921
Epoch 5/400
142/142 [==============================] - 6s 43ms/step - loss: 0.6928 - accuracy: 0.5153 - val_loss: 0.6923 - val_accuracy: 0.5207
Epoch 6/400
142/142 [==============================] - 6s 44ms/step - loss: 0.

...

   train_acc  train_f1  tune_acc   tune_f1  test_acc   test_f1
0   0.542364  0.542234  0.540139  0.537295  0.526994  0.525064
1   0.534762  0.533756  0.541808  0.541606  0.519326  0.516555
2   0.547543  0.546179  0.541286  0.541223  0.529185  0.527102
3   0.544623  0.543158  0.540347  0.540219  0.527046  0.521535
4   0.545670  0.544153  0.541234  0.540168  0.527411  0.519087
5   0.550463  0.546409  0.541182  0.540799  0.526420  0.502263
6   0.546717  0.546565  0.540034  0.532382  0.522351  0.497278
7   0.545615  0.545592  0.541234  0.534021  0.532680  0.532576
8   0.551730  0.546371  0.540347  0.540006  0.523760  0.506559
9   0.543632  0.542605  0.540504  0.540540  0.531636  0.531437
Attempt 11/12
Epoch 1/400
142/142 [==============================] - 41s 265ms/step - loss: 0.6956 - accuracy: 0.4970 - val_loss: 0.6931 - val_accuracy: 0.5089
Epoch 2/400
142/142 [==============================] - 37s 262ms/step - loss: 0.6940 - accuracy: 0.5003 - val_loss: 0.6953 - val_accuracy: 0.4911
E

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


    train_acc  train_f1  tune_acc   tune_f1  test_acc   test_f1
0    0.542364  0.542234  0.540139  0.537295  0.526994  0.525064
1    0.534762  0.533756  0.541808  0.541606  0.519326  0.516555
2    0.547543  0.546179  0.541286  0.541223  0.529185  0.527102
3    0.544623  0.543158  0.540347  0.540219  0.527046  0.521535
4    0.545670  0.544153  0.541234  0.540168  0.527411  0.519087
5    0.550463  0.546409  0.541182  0.540799  0.526420  0.502263
6    0.546717  0.546565  0.540034  0.532382  0.522351  0.497278
7    0.545615  0.545592  0.541234  0.534021  0.532680  0.532576
8    0.551730  0.546371  0.540347  0.540006  0.523760  0.506559
9    0.543632  0.542605  0.540504  0.540540  0.531636  0.531437
10   0.546772  0.546714  0.541234  0.539617  0.528715  0.528622
Attempt 12/12
Epoch 1/400
142/142 [==============================] - 40s 264ms/step - loss: 0.6952 - accuracy: 0.5018 - val_loss: 0.6930 - val_accuracy: 0.5058
Epoch 2/400
142/142 [==============================] - 36s 257ms/step - 

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


    train_acc  train_f1  tune_acc   tune_f1  test_acc   test_f1
0    0.542364  0.542234  0.540139  0.537295  0.526994  0.525064
1    0.534762  0.533756  0.541808  0.541606  0.519326  0.516555
2    0.547543  0.546179  0.541286  0.541223  0.529185  0.527102
3    0.544623  0.543158  0.540347  0.540219  0.527046  0.521535
4    0.545670  0.544153  0.541234  0.540168  0.527411  0.519087
5    0.550463  0.546409  0.541182  0.540799  0.526420  0.502263
6    0.546717  0.546565  0.540034  0.532382  0.522351  0.497278
7    0.545615  0.545592  0.541234  0.534021  0.532680  0.532576
8    0.551730  0.546371  0.540347  0.540006  0.523760  0.506559
9    0.543632  0.542605  0.540504  0.540540  0.531636  0.531437
10   0.546772  0.546714  0.541234  0.539617  0.528715  0.528622
11   0.551675  0.551389  0.542017  0.537753  0.528089  0.520694
time: 3h 40min 39s (started: 2021-06-12 15:06:30 +00:00)


In [30]:
r = pd.DataFrame(results)
r.to_csv(tune_results_file)
r

,attempt,layers,neurons,lr,train_time,epochs,train_acc,train_f1,tune_acc,tune_f1,test_acc,test_f1,tune_buy_percentage,test_buy_percentage
0,1,1,64,0.00100,231.948045,37,0.542364,0.542234,0.540139,0.537295,0.526994,0.525064,0.508946,0.50373
1,2,1,64,0.00010,250.118479,40,0.534762,0.533756,0.541808,0.541606,0.519326,0.516555,0.508946,0.50373
2,3,1,64,0.00001,300.638985,48,0.547543,0.546179,0.541286,0.541223,0.529185,0.527102,0.508946,0.50373
3,4,1,128,0.00100,598.633714,38,0.544623,0.543158,0.540347,0.540219,0.527046,0.521535,0.508946,0.50373
4,5,1,128,0.00010,693.680244,44,0.545670,0.544153,0.541234,0.540168,0.527411,0.519087,0.508946,0.50373
5,6,1,128,0.00001,883.107152,56,0.550463,0.546409,0.541182,0.540799,0.526420,0.502263,0.508946,0.50373
6,7,2,64,0.00100,1155.700086,82,0.546717,0.546565,0.540034,0.532382,0.522351,0.497278,0.508946,0.50373
7,8,2,64,0.00010,734.972144,52,0.545615,0.545592,0.541234,0.534021,0.532680,0.532576,0.508946,0.50373
8,9,2,64,0.00001,1165.579606,84,0.551730,0.546371,0.540347,0.540006,0.523760,0.506559,0.508946,0.50373
9,10,2,128,0.00100,2107.586008,57,0.543632,0.542605,0.540504,0.540540,0.531636,0.531437,0.508946,0.50373


time: 42.3 ms (started: 2021-06-12 18:47:10 +00:00)
